In [ ]:
import requests
import csv
import time

BASE_URL = "https://api.digikala.com/v1"
SEARCH_URL = f"{BASE_URL}/categories/mobile-phone/brands/samsung/search/"
COMMENT_URL = "{}/product/{}/comments/".format(BASE_URL, "{}")
HEADERS = {"User-Agent": "Mozilla/5.0"}


def get_products(page):
    url = f"{SEARCH_URL}?page={page}"
    r = requests.get(url, headers=HEADERS, timeout=10)
    if r.status_code != 200:
        print(f"⚠️ صفحه {page} خطا داد ({r.status_code})")
        return []
    data = r.json()
    return data.get("data", {}).get("products", [])


def get_comments(product_id):
    url = COMMENT_URL.format(product_id)
    try:
        r = requests.get(url, headers=HEADERS, timeout=10)
        r.raise_for_status()
    except requests.RequestException as e:
        print(f"⚠️ خطا در ارتباط با {url}: {e}")
        return []

    try:
        data = r.json()
    except ValueError:
        print(f"⚠️ پاسخ JSON معتبر نیست برای محصول {product_id}")
        return []

    # 🔹 مرحله‌ی تشخیص خودکار محل commentها
    comments = None
    if "comments" in data:
        comments = data["comments"]
    elif "data" in data and "comments" in data["data"]:
        comments = data["data"]["comments"]
    elif "data" in data and isinstance(data["data"], list):
        for item in data["data"]:
            if "comments" in item:
                comments = item["comments"]
                break
    elif "data" in data and isinstance(data["data"], dict): 
        comments = data["data"].get("comments", [])

    if not comments:
        print(f"⚠️ هیچ نظری در ساختار پاسخ برای محصول {product_id} پیدا نشد.")
        print("🔍 ساختار کلیدهای سطح بالا:", list(data.keys()))
        return []

    print(f"✅ {len(comments)} نظر برای محصول {product_id} یافت شد.")
    return comments


# --- فایل خروجی CSV ---
with open("samsung_comments.csv", "a", newline='', encoding="utf-8") as f:
    writer = csv.writer(f)

    # --- مرحله ۱: مرور صفحات ---
    for page in range(1, 101):
        print(f"📄 صفحه {page} در حال پردازش ...")
        products = get_products(page)
        if not products:
            print("⛔ صفحه خالی بود. پایان.")

        # --- مرحله ۲: دریافت کامنت‌های هر محصول ---
        for product in products:
            pid = product.get("id")
            pname = product.get("title_fa")
            comments = get_comments(pid)

            if not comments:
                print(f"  ❌ بدون نظر: {pname} {pid}")
                continue

            for c in comments:
                writer.writerow([
                    c.get("body"),
                    c.get("created_at"),
                    c.get("rate"),
                    pname,
                    c.get("user_name"),
                ])

            print(f"  ✅ {len(comments)} نظر از '  {pname}{pid}' ذخیره شد.")
            time.sleep(1)  # برای جلوگیری از مسدود شدن توسط سرور

        time.sleep(2)

print("🎉 تمام نظرات محصولات سامسونگ ذخیره شدند → samsung_comments.csv")